In [1]:
from csvb.ingest import apply_rule, sel_factory, ApplyOp, SelectOp, AOP, SOP, Rule, to_transactions, strip_currency
import csvb.ingest as csi
import json
import pathlib
import operator
import tomlkit as tml
from datetime import datetime

In [2]:
cfgpth = pathlib.Path.home() / ".config" / "csvb" / "config.toml"
with open(cfgpth, 'rb') as f:
    cfg = tml.load(f)

rulespth = pathlib.Path().home() / pathlib.Path(cfg["paths"]["rules"])
print(rulespth)
p = pathlib.Path(cfg["paths"]["rules"])

/Users/kdavis10/als/csvbdata/rules


In [3]:
dicts= [dict(select=dict(op="contains", column="Merchant", b="Banana Republic"),        apply=dict(op="assign", column="From", b="Expenses:Clothing")),
          dict(select=dict(op="contains", column="Merchant", b="Apple Services"),         apply=dict(op="assign", column="From", b="Expenses:Subscription")),
          dict(select=dict(op="contains", column="Merchant", b="Phillips 66"),            apply=dict(op="assign", column="From", b="Expenses:Auto:Gas")),
          dict(select=dict(op="contains", column="Merchant", b="Ridge Wallet"),           apply=dict(op="assign", column="From", b="Expenses:Gear")),
          dict(select=dict(op="contains", column="Merchant", b="Martin's Supermarket"),   apply=dict(op="assign", column="From", b="Expenses:Groceries")),
          dict(select=dict(op="contains", column="Merchant", b="Apple Cash Payment"),     apply=dict(op="assign", column="From", b="Income:Apple Cash")),
          dict(select=dict(op="contains", column="Merchant", b="Hulu"),                   apply=dict(op="assign", column="From", b="Expenses:Entertainment")),
          dict(select=dict(op="contains", column="Merchant", b="Trade Coffee CO"),        apply=dict(op="assign", column="From", b="Expenses:Groceries")),
          dict(select=dict(op="contains", column="Merchant", b="Beacon Med"),             apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
          dict(select=dict(op="contains", column="Merchant", b="South Bend Clinic"),      apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
          dict(select=dict(op="contains", column="Merchant", b="WMMH"),                   apply=dict(op="assign", column="From", b="Expenses:Entertainment:Self")),
          dict(select=dict(op="contains", column="Merchant", b="Account Ending In 8785"), apply=dict(op="assign", column="From", b="Assets:1st Source:Checking")),
          dict(select=dict(op="contains", column="Merchant", b="bookshop.org"),           apply=dict(op="assign", column="From", b="Expenses:Study")),
          dict(select=dict(op="contains", column="Merchant", b="Patreon"),                apply=dict(op="assign", column="From", b="Expenses:Entertainment")),
          dict(select=dict(op="contains", column="Merchant", b="Monthly Installments"),   apply=dict(op="assign", column="From", b="Liabilities:Apple:Installments")),
          dict(select=dict(op="contains", column="Category", b="Tolls"),                  apply=dict(op="assign", column="From", b="Expenses:Auto:Tolls")),
          dict(select=dict(op="contains", column="Merchant", b="Kobo"),                   apply=dict(op="assign", column="From", b="Expenses:Entertainment:Reading")),
          dict(select=dict(op="contains", column="Merchant", b="Vtsup.com"),              apply=dict(op="assign", column="From", b="Expenses:Entertainment:Self")),
          dict(select=dict(op="contains", column="Merchant", b="City Of South Bend"),     apply=dict(op="assign", column="From", b="Expenses:Utilities")),
          dict(select=dict(op="contains", column="Merchant", b="Andrasi"),                apply=dict(op="assign", column="From", b="Expenses:Home:Cleaning")),
          dict(select=dict(op="contains", column="Merchant", b="Parlevel Texas"),         apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
          dict(select=dict(op="contains", column="Merchant", b="Gannett Newsprpr"),       apply=dict(op="assign", column="From", b="Expenses:Utilities")),
          dict(select=dict(op="contains", column="Description", b="ABEBOOKS"),            apply=dict(op="assign", column="From", b="Expenses:Study"))]

rules = []
for d in dicts:
    rules.append(Rule(**d))

doc = tml.document()
doc.add("title", "A list of rules for processing Apple Credit Card Transactions.")
tmlrules = tml.aot()
for r in rules:
    tmlrules.append(r.to_TOML())

doc.add("rules", tmlrules)
with open(rulespth / "aplrules.toml", 'w') as f:
    tml.dump(doc, f)




ApplyOp(op='assign', column='From', b='Expenses:Clothing') op assign
ApplyOp(op='assign', column='From', b='Expenses:Clothing') column From
ApplyOp(op='assign', column='From', b='Expenses:Clothing') b Expenses:Clothing
SelectOp(op='contains', column='Merchant', a=None, b='Banana Republic') op contains
SelectOp(op='contains', column='Merchant', a=None, b='Banana Republic') column Merchant
SelectOp(op='contains', column='Merchant', a=None, b='Banana Republic') a None
SelectOp(op='contains', column='Merchant', a=None, b='Banana Republic') b Banana Republic
ApplyOp(op='assign', column='From', b='Expenses:Subscription') op assign
ApplyOp(op='assign', column='From', b='Expenses:Subscription') column From
ApplyOp(op='assign', column='From', b='Expenses:Subscription') b Expenses:Subscription
SelectOp(op='contains', column='Merchant', a=None, b='Apple Services') op contains
SelectOp(op='contains', column='Merchant', a=None, b='Apple Services') column Merchant
SelectOp(op='contains', column='Mer

In [4]:
dicts = [dict(select=[dict(op="contains", column="Description", b="crossfit south"),
                      dict(op=operator.ne,column="Amount", b=-225.00)], apply=dict(op="assign", column="From", b="Expenses:Groceries")),
         dict(select=[dict(op="contains", column="Description", b="crossfit south"),
                      dict(op=operator.eq,column="Amount", b=-225.00)], apply=dict(op="assign", column="From", b="Expenses:Groceries")),
             dict(select=dict(op="contains", column="Description", b="ND PAYROLL"), apply=dict(op="assign", column="From", b="Income:Notre Dame:Regular")),
             dict(select=dict(op="contains", column="Description", b="INTEREST PAYMENT"), apply=dict(op="assign", column="From", b="Income:Interest")),
             dict(select=dict(op="contains", column="Description", b="NIPSCO"), apply=dict(op="assign", column="From", b="Expenses:Utilities")),
             dict(select=dict(op="contains", column="Description", b="AEP"), apply=dict(op="assign", column="From", b="Expenses:Utilities")),
             dict(select=dict(op="contains", column="Description", b="ALLY BANK \$TRANSFER"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),
             dict(select=dict(op="contains", column="Description", b="VERIZON"), apply=dict(op="assign", column="From", b="Expenses:Utilities")),
             dict(select=[dict(op="contains", column="Description", b="VENMO PAYMENT"),
                          dict(op=operator.eq, column="Date", b=datetime(2024,1,3))], apply=dict(op="assign", column="From", b="Expenses:Gifts")),
             dict(select=dict(op="contains", column="Description", b="KEITH W DAVIS P2P"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),
             dict(select=dict(op="contains", column="Description", b="DOVENMUEHLE MTG MORTG PYMT"), apply=dict(op="assign", column="From", b="Expenses:Home:Mortgage"))
            ]

rules = []
for d in dicts:
    rules.append(Rule(**d))

doc = tml.document()
doc.add("title", "A list of rules for processing First Source Checking Transactions.")
tmlrules = tml.aot()
for r in rules:
    tmlrules.append(r.to_TOML())

doc.add("rules", tmlrules)
with open(rulespth / "fstrules.toml", 'w') as f:
    tml.dump(doc, f)


ApplyOp(op='assign', column='From', b='Expenses:Groceries') op assign
ApplyOp(op='assign', column='From', b='Expenses:Groceries') column From
ApplyOp(op='assign', column='From', b='Expenses:Groceries') b Expenses:Groceries
SelectOp(op='contains', column='Description', a=None, b='crossfit south') op contains
SelectOp(op='contains', column='Description', a=None, b='crossfit south') column Description
SelectOp(op='contains', column='Description', a=None, b='crossfit south') a None
SelectOp(op='contains', column='Description', a=None, b='crossfit south') b crossfit south
SelectOp(op=<built-in function ne>, column='Amount', a=None, b=-225.0) op <built-in function ne>
SelectOp(op=<built-in function ne>, column='Amount', a=None, b=-225.0) column Amount
SelectOp(op=<built-in function ne>, column='Amount', a=None, b=-225.0) a None
SelectOp(op=<built-in function ne>, column='Amount', a=None, b=-225.0) b -225.0
ApplyOp(op='assign', column='From', b='Expenses:Groceries') op assign
ApplyOp(op='assi

<>:9: SyntaxWarning: invalid escape sequence '\$'
<>:9: SyntaxWarning: invalid escape sequence '\$'
/var/folders/42/qqsltqsj6fl30wl4xm41gs4m0000gn/T/ipykernel_13722/3207374124.py:9: SyntaxWarning: invalid escape sequence '\$'
  dict(select=dict(op="contains", column="Description", b="ALLY BANK \$TRANSFER"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),


In [5]:
dicts = [dict(select=dict(op="contains", column="Description", b="transfer"), apply=dict(op="assign", column="From", b="Transfer:Ally:Savings")),
             dict(select=dict(op="contains", column="Description", b="Interest Paid"), apply=dict(op="assign", column="From", b="Income:Interest"))]

rules = []
for d in dicts:
    rules.append(Rule(**d))

doc = tml.document()
doc.add("title", "A list of rules for processing First Source Checking Transactions.")
tmlrules = tml.aot()
for r in rules:
    tmlrules.append(r.to_TOML())

doc.add("rules", tmlrules)
with open(rulespth / "allyrules.toml", 'w') as f:
    tml.dump(doc, f)


ApplyOp(op='assign', column='From', b='Transfer:Ally:Savings') op assign
ApplyOp(op='assign', column='From', b='Transfer:Ally:Savings') column From
ApplyOp(op='assign', column='From', b='Transfer:Ally:Savings') b Transfer:Ally:Savings
SelectOp(op='contains', column='Description', a=None, b='transfer') op contains
SelectOp(op='contains', column='Description', a=None, b='transfer') column Description
SelectOp(op='contains', column='Description', a=None, b='transfer') a None
SelectOp(op='contains', column='Description', a=None, b='transfer') b transfer
ApplyOp(op='assign', column='From', b='Income:Interest') op assign
ApplyOp(op='assign', column='From', b='Income:Interest') column From
ApplyOp(op='assign', column='From', b='Income:Interest') b Income:Interest
SelectOp(op='contains', column='Description', a=None, b='Interest Paid') op contains
SelectOp(op='contains', column='Description', a=None, b='Interest Paid') column Description
SelectOp(op='contains', column='Description', a=None, b=

In [6]:
dicts = [dict(select=dict(op="contains", column="Description", b="AMZN"),        apply=dict(op="assign", column="From", b="Expenses:Home")),
               dict(select=dict(op="contains", column="Description", b="Netflix.com"),        apply=dict(op="assign", column="From", b="Expenses:Subscriptions")),
               dict(select=dict(op="contains", column="Description", b="Amazon Prime"),        apply=dict(op="assign", column="From", b="Expenses:Subscriptions")),
               dict(select=dict(op="contains", column="Description", b="CHEWY"),        apply=dict(op="assign", column="From", b="Expenses:Pet")),
               dict(select=dict(op="contains", column="Description", b="CHIPOTLE"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
               dict(select=dict(op="contains", column="Description", b="LA ESPERANZA"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
               dict(select=dict(op="contains", column="Description", b="CHICORY"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),
               dict(select=dict(op="contains", column="Description", b="PREMISE"),        apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
               dict(select=dict(op="contains", column="Description", b="WALGREENS"),        apply=dict(op="assign", column="From", b="Expenses:Medical:Body")),
               dict(select=dict(op="contains", column="Description", b="NYTimes"),        apply=dict(op="assign", column="From", b="Expenses:Subscriptions")),
               dict(select=dict(op="contains", column="Description", b="COMCAST"),        apply=dict(op="assign", column="From", b="Expenses:Utilities")),
               dict(select=dict(op="contains", column="Description", b="MARTIN S"),        apply=dict(op="assign", column="From", b="Expenses:Groceries")),
               dict(select=dict(op="contains", column="Description", b="FEATHERGILL"),        apply=dict(op="assign", column="From", b="Expenses:Medical:Therapy")),
               dict(select=dict(op="contains", column="Description", b="PHILLIPS 66"),        apply=dict(op="assign", column="From", b="Expenses:Auto:Fuel")),
               dict(select=dict(op="contains", column="Description", b="JESUS"),        apply=dict(op="assign", column="From", b="Expenses:Restaurants")),               
         ]

rules = []
for d in dicts:
    rules.append(Rule(**d))

doc = tml.document()
doc.add("title", "A list of rules for processing First Source Checking Transactions.")
tmlrules = tml.aot()
for r in rules:
    tmlrules.append(r.to_TOML())

doc.add("rules", tmlrules)
with open(rulespth / "ndfcurules.toml", 'w') as f:
    tml.dump(doc, f)

    

ApplyOp(op='assign', column='From', b='Expenses:Home') op assign
ApplyOp(op='assign', column='From', b='Expenses:Home') column From
ApplyOp(op='assign', column='From', b='Expenses:Home') b Expenses:Home
SelectOp(op='contains', column='Description', a=None, b='AMZN') op contains
SelectOp(op='contains', column='Description', a=None, b='AMZN') column Description
SelectOp(op='contains', column='Description', a=None, b='AMZN') a None
SelectOp(op='contains', column='Description', a=None, b='AMZN') b AMZN
ApplyOp(op='assign', column='From', b='Expenses:Subscriptions') op assign
ApplyOp(op='assign', column='From', b='Expenses:Subscriptions') column From
ApplyOp(op='assign', column='From', b='Expenses:Subscriptions') b Expenses:Subscriptions
SelectOp(op='contains', column='Description', a=None, b='Netflix.com') op contains
SelectOp(op='contains', column='Description', a=None, b='Netflix.com') column Description
SelectOp(op='contains', column='Description', a=None, b='Netflix.com') a None
Select

In [1]:
with open(rulespth / "ndfcurules.toml", 'r') as f:
    ndfcur = tml.load(f)

NameError: name 'rulespth' is not defined